<a href="https://colab.research.google.com/github/mohamedahmedsaadahmed77/Sarcasm-Detection/blob/main/English_Sarcasm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/mohamedahmedsaadahmed77/Sarcasm_Detection.git

Cloning into 'Sarcasm_Detection'...
remote: Enumerating objects: 16, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 16 (delta 7), reused 5 (delta 0), pack-reused 0
Unpacking objects: 100% (16/16), done.


In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

import re
import string
import csv


import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:
def preprocessing(sentences):
  lowercase = lambda x : x.lower()
  sentences = list(map(lowercase, sentences))

  remove_punctuation = lambda x : re.sub('[%s]' % re.escape(string.punctuation), ' ', x)
  sentences = list(map(remove_punctuation, sentences))
  
  remove_digits = lambda x : re.sub('\w*\d\w*', ' ', x)
  sentences = list(map(remove_digits, sentences))
  
  convert_to_tokens = lambda x : word_tokenize(x)
  tokens = list(map(convert_to_tokens, sentences))

  stemmer = PorterStemmer()
  stem_tokens = lambda x : [stemmer.stem(w) for w in x]
  stemmed_tokens = list(map(stem_tokens, tokens))
  
  stop_words = set(stopwords.words('english'))
  remove_stopwords = lambda x : [w for w in x if not w in stop_words]
  stemmed_tokens = list(map(remove_stopwords, stemmed_tokens))
  
  return stemmed_tokens

In [4]:
class TextNBClassifier:
    def __init__(self):
        pass
    
    def count_words_freq(self, xs, ys):
      freqs = {}
      for words, y in zip(xs, ys):
          for word in words:
              pair = (word,y)
              if pair in freqs:
                  freqs[pair] += 1
              else:
                  freqs[pair] = 1
      return freqs

    def lookup(self, freqs, word, is_sarcastic):
      if (word, is_sarcastic) in freqs:
        return freqs[(word, is_sarcastic)]
      else:
        return 0
      return freq

    def fit(self, train_x, train_y):
      freqs = self.count_words_freq(train_x, train_y)
      self.loglikelihood = {}
      self.logprior = 0

      vocab = set([pair[0] for pair in freqs.keys()])
      V = len(vocab)

      N_pos = N_neg = V_pos = V_neg = 0
      for pair in freqs.keys():
          if pair[1] > 0:
              V_pos += 1
              N_pos += freqs[pair]
          else:
              V_neg += 1
              N_neg += freqs[pair]
      D = len(train_y)
      
      D_pos = (len(list(filter(lambda x: x > 0, train_y))))
      D_neg = (len(list(filter(lambda x: x <= 0, train_y))))

      logprior = np.log(D_pos) - np.log(D_neg)

      for word in vocab:
          freq_pos = self.lookup(freqs,word,1)
          freq_neg = self.lookup(freqs,word,0)

          p_w_pos = (freq_pos + 1) / (N_pos + V)
          p_w_neg = (freq_neg + 1) / (N_neg + V)

          self.loglikelihood[word] = np.log(p_w_pos/p_w_neg)
      return self

    def predict_per_record(self, word_l):
      p = 0
      p += self.logprior

      for word in word_l:
        if word in self.loglikelihood:
              p += self.loglikelihood[word]
      return p

    def predict(self, test_x, test_y):
      self.accuracy = 0
      y_hats = []
      for word_l in test_x:
          if self.predict_per_record(word_l) > 0:
              y_hat_i = 1
          else:
              y_hat_i = 0
          y_hats.append(y_hat_i)
      error = np.mean(np.absolute(y_hats-test_y))
      self.accuracy = 1-error    
      return self

In [5]:
data = pd.read_csv ('/content/Sarcasm_Detection/English_Sarcasm.csv')
print(data.shape)
data.head()

(28619, 3)


,is_sarcastic,headline,article_link
0,1,thirtysomething scientists unveil doomsday clo...,https://www.theonion.com/thirtysomething-scien...
1,0,dem rep. totally nails why congress is falling...,https://www.huffingtonpost.com/entry/donna-edw...
2,0,eat your veggies: 9 deliciously different recipes,https://www.huffingtonpost.com/entry/eat-your-...
3,1,inclement weather prevents liar from getting t...,https://local.theonion.com/inclement-weather-p...
4,1,mother comes pretty close to using word 'strea...,https://www.theonion.com/mother-comes-pretty-c...


In [6]:
x_train, x_test, Y_train, Y_test = train_test_split(preprocessing(data['headline']), data['is_sarcastic'], test_size=0.2, random_state=42)

In [7]:
model = TextNBClassifier()
model.fit(x_train, Y_train)
model.predict(x_train, Y_train)
print("Training Accuracy : %.2f"%(model.accuracy * 100))

Training Accuracy : 87.75


In [8]:
model.predict(x_test, Y_test)
print("Test Accuracy : %.2f"%(model.accuracy * 100))

Test Accuracy : 79.89
